# Story

In [1]:
from pathselection import joinpathselection
from integrating_civilizer import *
import qgrid
# import pandas as pd
data_path = "/Users/qahtanaa/git/aurum-datadiscovery/MIT_Data/"
model_path = "test/MIT_Model/"
(api, reporting, corpus, tables_dfs, dirty_cells) = init_component(data_path, model_path)


Loading: *test/MIT_Model/*

### Help Menu

You can use the system through an **API** object. API objects are returnedby the *init_system* function, so you can get one by doing:

***your_api_object = init_system('path_to_stored_model')***

Once you have access to an API object there are a few concepts that are useful to use the API. **content** refers to actual values of a given field. For example, if you have a table with an attribute called __Name__ and values *Olu, Mike, Sam*, content refers to the actual values, e.g. Mike, Sam, Olu.

**schema** refers to the name of a given field. In the previous example, schema refers to the word__Name__ as that's how the field is called.

Finally, **entity** refers to the *semantic type* of the content. This is in experimental state. For the previous example it would return *'person'* as that's what those names refer to.

Certain functions require a *field* as input. In general a field is specified by the source name (e.g. table name) and the field name (e.g. attribute name). For example, if we are interested in finding content similar to the one of the attribute *year* in the table *Employee* we can provide the field in the following way:

field = ('Employee', 'year') # field = [<source_name>, <field_name>)
Took 0.050714969635009766 to load all data


In [2]:
cleanliness_estimation(data_path)

In [3]:
TC = display_violations()

EmptyDataError: No columns to parse from file

# JP Selection

We use pandas to display the contents of a given table so that the user can take decision if he is interested in this table or not. The user can control the number of displayed records using pandas.set_option()

In [ ]:
t1 = "Hr_org_unit.csv"
pd.set_option('display.max_rows', 100)
df = get_table(t1, tables_dfs)
df

Now, we display (X) records, (X/2) from the beginning of the table and (X/2) from the end of the table. The records are displayed with missing cells highlighted in "YELLOW" and the dirty cells highlighted in "RED"

In [ ]:
pd.set_option('display.max_rows', 100)
S = load_table_cleanliness(t1, tables_dfs, dirty_cells)
S

The user can also select another table to discover the possible join paths between the new table and table (t1)

In [ ]:
t2 = "Hr_org_unit_new.csv"

In [ ]:
JPs, formatted_JPs = joinPaths(api, t1, t2)
print(formatted_JPs)

There are different join paths! Does anyone work? What are the differences?

# 3 - Selecting Join Paths

### There are different join paths! Which join path should I use? What are the differences?

The join path selection module selects one join path from perhaps many, which maximizes the estimated benefit in terms of the cardinalities and the cleanliness


In [ ]:
projections = dict()
projections['Hr_org_unit.csv'] = ['Hr Org Unit Key','Hr Org Unit Id','Hr Org Unit Title','Hr Org Unit Level','Hr Department Id']
projections['Hr_org_unit_new.csv'] = ['Hr Org Unit Key','Hr Org Unit Id','Hr Org Unit Title','Hr Org Unit Level']

jps, JP_with_score = join_path_selection(api, JPs, projections, corpus)
qgrid.show_grid(JP_with_score, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100, 'show_toolbar' : True})

# 4 - Cleaning the Join Path with a Budget 
### If I have some budget, which cell to clean first?

The join path cleaning module uses a given budget of B user repairing actions to maximize the quality of the view by cleaning the most impact cells in the source table first

In [ ]:
budget = 10
join_path = 7
res, df = clean_with_budget(jps, corpus, budget, join_path, dirty_cells)
df

In [ ]:
res, df = clean_with_budget_old(jps, corpus, budget, join_path)
df

In [ ]:
if len(df) > 0:
    dfs = peek_results(df, tables_dfs)
    for el in dfs:
        qgrid.show_grid(el, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100, 'show_toolbar' : True})
else:
    print('Nothing needs to be cleaned ..')

In [ ]:
save_modifications(df, dfs, tables_dfs, data_path, dirty_cells)